In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import innvestigate
import os
tf.compat.v1.disable_eager_execution()

In [9]:
from __future__ import annotations

import inspect

import tensorflow.keras.backend as kbackend
import tensorflow.keras.layers as klayers

import innvestigate.analyzer.relevance_based.relevance_rule as rrule
import innvestigate.analyzer.relevance_based.utils as rutils
import innvestigate.analyzer.relevance_based.relevance_analyzer as ianalyzer
import innvestigate.analyzer.relevance_based.relevance_rule as irules
import innvestigate.backend as ibackend
import innvestigate.backend.checks as ichecks
import innvestigate.backend.graph as igraph
from innvestigate.analyzer.network_base import AnalyzerNetworkBase
from innvestigate.analyzer.reverse_base import ReverseAnalyzerBase
from innvestigate.analyzer import LRP
from innvestigate.backend.types import Layer, LayerCheck, Model, OptionalList, Tensor

In [10]:
loadedModel = keras.models.load_model("CNN_model")


2023-06-03 13:21:30.530885: W tensorflow/c/c_api.cc:291] Operation '{name:'AssignVariableOp_26' id:529 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_26}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](count_3, Identity_26)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [11]:
noSoftMaxModel = innvestigate.model_wo_softmax(loadedModel)

In [12]:
analyzer = LRP(noSoftMaxModel,rule="Z,Epsilon,Alpha1Beta0",input_layer_rule="Bounded",until_layer_idx=5,until_layer_rule="Gamma",bn_layer_rule="Z")

In [25]:
folders = ["00","01","02","03","04","05","06","07","08","09"]
CATEGORIES = ["01_palm", '02_l','03_fist','04_fist_moved','05_thumb','06_index','07_ok','08_palm_moved','09_c','10_down']
IMG_SIZE = 50

# paths for dataset
data_path = "../kaggle_data"

# Loading the images and their class(0 - 9)
# Loading the images and their class(0 - 9)
image_data = []
for dr in folders:
    for category in CATEGORIES:
        class_index = CATEGORIES.index(category)
        path = os.path.join(data_path, dr, category)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                image_data.append([cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE)), class_index])
            except Exception as e:
                pass


In [27]:
image_data[0]

IndexError: list index out of range

In [14]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import densenet

preprocess = vgg16.preprocess_input

In [269]:
ppin=image_data[0]
pp = vgg16.preprocess_input(ppin[0])
a = analyzer.analyze(pp)

Analyze image

In [271]:
# Add batch axis and preprocess
x = preprocess(ppin)
# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(x)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))
# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))

AttributeError: 'numpy.ndarray' object has no attribute 'layers'

Create Analyzer
#(no maxpooling and input layer)
[ ]

In [60]:
class ruleHandler(ianalyzer.LRP):
    
    allLayer = ["2_block1_conv3","4_block2_conv4","8_block3_conv5","12_dense_2","13_dense_3"]
    flatLayer = ["0_input_layer", "2_block1_conv3"]
    alpha1Beta0Layer = ["4_block2_conv4","8_block3_conv5"]
    epsilonLayer = ["12_dense_2","13_dense_3"]
    lrpZeroLayer = ["12_dense_2","13_dense_3"]
    
    def __init__(self, model, neuron_selection_mode="index", *args, **kwargs):
        print("Selected Costume Analyzer (CostumeVGG16)")
        _neuron_selection_mode = neuron_selection_mode

        rules = [
            (is_alpha1_beta0_rule_layer, irules.Alpha1Beta0Rule),
            (is_epsilon_rule_layer, irules.ZRule),
            (is_flat_rule_layer, irules.FlatRule),
            (is_default, irules.ZRule) #always last or every layer will be assigned this rule
        ]
        super(ruleHandler, self).__init__(model, *args, neuron_selection_mode=_neuron_selection_mode, rule=rules, input_layer_rule=irules.FlatRule, **kwargs)
    
   
    
    def is_alpha1_beta0_rule_layer(layer, reverse_state):
        print("Alpha: " + str(layer.name) + " -> " + str(layer.name in alpha1Beta0Layer))
        return layer.name in alpha1Beta0Layer


    def is_epsilon_rule_layer(layer, reverse_state):
        print("Epsilon: " + str(layer.name) + " -> " + str(layer.name in epsilonLayer))
        return layer.name in epsilonLayer


    def is_flat_rule_layer(layer, reverse_state):
        print("Flat: " + str(layer.name) + " -> " + str(layer.name in flatLayer))
        return layer.name in flatLayer


    def is_default(layer, reverse_state):
        print("Default: " + str(layer.name))
        return True
    
 

In [ ]:
class lrpHandler:
    R_MAP_GENERATION_MODE = [
        "ALL", # generate relevance map for every AU for every image
        "ONLY RELEVANT", #only generate relvance map is AU is either in ground truth or predicted by network
        "ONLY PREDICTED" #only if network predicts AU is active
    ]

    def __init__(self, app_state: AppState, on_complete: Callable = None, progress_update: Callable[[int, str], None] = None):
        self.app_state = app_state
        self.progress_update = progress_update
        self.on_complete = on_complete

        # Get set each time at before analysis
        self.sequences = None
        self.model = None
        self.modelPath = None
        self.modelOutput = None
        self.analyzer = None
        self.outPath = None
        self.relevanceAnchorPath = None
        self.rMapGenerationMode = "ALL"
   
    